In [14]:
import crypto
import sys
sys.modules['Crypto'] = crypto
from Crypto.Cipher import AES

In [42]:
class AES_Cihper:
    def __init__(self, key, mode):
        self.mode = mode
        self.key = bytes.fromhex(key.decode('us-ascii'))
        self.cipher = AES.new(self.key, AES.MODE_ECB)

    def text_padding(self, plaintext, padding_length):
        offset_padding = len(plaintext)
        plaintext = list(plaintext)
        for i in range(offset_padding, offset_padding + padding_length):
            plaintext.append(padding_length)
        
        return bytes(plaintext)
    
    def eliminate_padding_and_print(self, text, eliminate):
        output = []
        if(eliminate == True):
            padding_size = text[len(text) - 1][15]
            sum = 0
            for i in range(15, 15 - padding_size, -1):
                sum += text[len(text) - 1][i]
            if(sum/padding_size != padding_size):
                padding_size = 0
            else:
                text[len(text) - 1] = text[len(text) - 1][0:16-padding_size]
        for i in range(0, len(text)):
            print(text[i].hex(), end ="")
            output += text[i]
        print('\n')
        return bytes(output).hex()
    
    def bxor(self,b1, b2):
        result = bytearray(b2)
        result2 = bytearray(b1)
        for i in range (len(bytearray(b2))-1, -1, -1):
            result[i] ^= result2[i]
        return bytes(result)

    def encrypt(self, plaintext, iv, eliminate_padding):
        iv  = bytes.fromhex(iv.decode('us-ascii'))
        iv_len = len(iv)
        plaintext = bytes.fromhex(plaintext.decode('us-ascii'))
        padding_size = 16 - (len(plaintext) % 16)
        ciphertext = []
        block = []
        xor_result = []
        
        if("CBC" == self.mode):
            plaintext = self.text_padding(plaintext, padding_size)
            for i in range(0, int(len(plaintext)/16)):
                block.append(bytes(list(plaintext)[i*16:(i+1)*16]))
                if (i==0):
                    xor_result = self.bxor(block[i], iv)
                else:
                    xor_result =  self.bxor(block[i], ciphertext[i-1])
                ciphertext.append(self.cipher.encrypt(xor_result))
            return self.eliminate_padding_and_print(ciphertext, eliminate_padding)
        
        elif("CTR" == self.mode):
            no_of_blocks = int(len(ciphertext)/16) + (1 if padding_size != 0 else 0)
            padding_size = (len(ciphertext) % 16)

            for i in range(0, no_of_blocks):
                block.append(bytes(list(plaintext)[i*16:(i+1)*16]))
                ciphertext.append(self.bxor(block[i], self.cipher.encrypt(iv)))
                iv = int.from_bytes(iv, 'big')
                iv = iv + 1
                iv = iv.to_bytes(iv_len, 'big')
            return self.eliminate_padding_and_print(ciphertext, eliminate_padding)
        else:
            print("Cipher mode not supported")
            
    def decrypt(self, ciphertext, eliminate_padding):
        ciphertext = bytes.fromhex(ciphertext.decode('us-ascii'))
        plaintext = []
        block = []
        iv = []
        if("CBC" == self.mode):
            block.append(bytes(list(ciphertext)[0:16])) #append IV
            for i in range(1, int(len(ciphertext)/16)):
                block.append(bytes(list(ciphertext)[i*16:(i+1)*16]))
                plaintext.append(self.bxor(self.cipher.decrypt(block[i]), block[i-1]))
            return self.eliminate_padding_and_print(plaintext, eliminate_padding)
        elif("CTR" == self.mode):
            padding_size = (len(ciphertext) % 16)
            iv = bytes(list(ciphertext)[0:16]) #get IV
            iv_len = len(iv)
            no_of_blocks = int(len(ciphertext)/16) + (1 if padding_size != 0 else 0)
            for i in range(1, no_of_blocks):
                block.append(bytes(list(ciphertext)[i*16:(i+1)*16]))
                plaintext.append(self.bxor(self.cipher.encrypt(iv), block[i-1]))
                iv = int.from_bytes(iv, 'big')
                iv = iv + 1
                iv = iv.to_bytes(iv_len, 'big')
            return self.eliminate_padding_and_print(plaintext, eliminate_padding)
        else:
            print("Cipher mode not supported")

In [43]:
##TESTS
key1 = b'140b41b22a29beb4061bda66b6747e14'
iv1  = b'120b41b22429beb40655da66b6747e14'
plaintext1 = b'4ca00ff4c8982323d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81'

cipher_1 = AES_Cihper(key1, "CBC")
print("\nEncrypt CBC:")
cipher_1.encrypt(plaintext1, iv1, False)


ciphertext1 = iv1 + b'16d7d08c1d1ba69bf14c553bf664f0af704e7f0aac0d43e4fa184713cbec5a2668b05d555839933a4f4179cea642c6b7e298b19f934107d0ee828fdcf6ae7ca7a89c782c0518f41d752fba20921259ea '
print("\nDecrypt CBC :")
dec_plaintext1 = cipher_1.decrypt(ciphertext1, True)
print("\nPlaintext == decrypted plaintext: ", dec_plaintext1 == plaintext1.decode())



cipher_2 = AES_Cihper(key1, "CTR")
print("\nEncrypt CTR:")
cipher_2.encrypt(plaintext1, iv1, False)

ciphertext2 = iv1 + b'3dd1038057d284022977ac43102d2e2300cae8ba3cdbd5a86098fe8cc7180b8ac28a76823e9c1942cd0ce0c05533f7b73ae2be36845f717a5f499763a0440cd8a3b03b91bab1c2e3bbe859ddd0c86c73'
print("\nDecrypt CTR:")
dec_plaintext2 = cipher_2.decrypt(ciphertext2, True)
print("\nPlaintext == decrypted plaintext: ", dec_plaintext2 == plaintext1.decode())


Encrypt CBC:
16d7d08c1d1ba69bf14c553bf664f0af704e7f0aac0d43e4fa184713cbec5a2668b05d555839933a4f4179cea642c6b7e298b19f934107d0ee828fdcf6ae7ca7a89c782c0518f41d752fba20921259ea


Decrypt CBC :
4ca00ff4c8982323d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81


Plaintext == decrypted plaintext:  True

Encrypt CTR:
3dd1038057d284022977ac43102d2e23


Decrypt CTR:
4ca00ff4c8982323d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81


Plaintext == decrypted plaintext:  True


In [44]:
### TO DECRYPT

#Q1
q1_key = b'140b41b22a29beb4061bda66b6747e14'
q1_ciphertext = b'4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81'
q1_cipher = AES_Cihper(q1_key, "CBC")
print("Q1: ")
q1_cipher.decrypt(q1_ciphertext, True)
#A: Basic CBC mode encryption needs padding.

#Q2
q2_key = b'140b41b22a29beb4061bda66b6747e14'
q2_ciphertext = b'5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253'
q2_cipher = AES_Cihper(q2_key, "CBC")
print("Q2: ")
q2_cipher.decrypt(q2_ciphertext, True)
#A: Our implementation uses rand. IV

#Q3
q3_key = b'36f18357be4dbd77f050515c73fcf9f2'
q3_ciphertext = b'69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329'
q3_cipher = AES_Cihper(q3_key, "CTR")
print("Q3: ")
q3_cipher.decrypt(q3_ciphertext, False)
#A CTR mode lets you build a stream cipher from a block cipher.


#Q4
q4_key = b'36f18357be4dbd77f050515c73fcf9f2'
q4_ciphertext = b'770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451'
q4_cipher = AES_Cihper(q4_key, "CTR")
print("Q4: ")
dummy = q4_cipher.decrypt(q4_ciphertext, False)
#A Always avoid the two time pad!

Q1: 
426173696320434243206d6f646520656e6372797074696f6e206e656564732070616464696e672e

Q2: 
4f757220696d706c656d656e746174696f6e20757365732072616e642e204956

Q3: 
435452206d6f6465206c65747320796f75206275696c6420612073747265616d206369706865722066726f6d206120626c6f636b206369706865722e

Q4: 
416c776179732061766f6964207468652074776f2074696d652070616421

